## Libraries

In [9]:
# Data Handling
import pandas as pd
import numpy as np

# Data Split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
seed = 42
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

# Modeling
from sklearn.model_selection import cross_val_score
#  - Bagging,models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

#  - Boosting models
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

# Evaluation
from sklearn.metrics import log_loss

## Load Data

In [20]:
train_features = pd.read_csv('./data/train_features.csv')
test_features = pd.read_csv('./data/test_features.csv')
target = pd.read_csv('./data/target.csv').LABEL
print(train_features.shape, test_features.shape, target.shape)

(184172, 75) (78932, 75) (184172,)


 - Hold Out Method

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(train_features, target, test_size = 0.3, random_state = seed)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(128920, 75) (55252, 75) (128920,) (55252,)


## Ensemble


In [23]:
rf_clf = RandomForestClassifier(random_state=seed, n_jobs=-1)
extra_clf = ExtraTreesClassifier(random_state=seed, n_jobs=-1)
gbm_clf = GradientBoostingClassifier(random_state=seed)
xgb_clf = XGBClassifier(random_state=seed, n_jobs=-1)
lgb_clf = LGBMClassifier(random_state=seed, n_jobs=-1)

models = [extra_clf, xgb_clf, lgb_clf]

estimators = [(model.__class__.__name__, model) for model in models]
estimators

[('ExtraTreesClassifier', ExtraTreesClassifier(n_jobs=-1, random_state=42)),
 ('XGBClassifier',
  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None, gamma=None,
                gpu_id=None, importance_type='gain', interaction_constraints=None,
                learning_rate=None, max_delta_step=None, max_depth=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=-1, num_parallel_tree=None,
                random_state=42, reg_alpha=None, reg_lambda=None,
                scale_pos_weight=None, subsample=None, tree_method=None,
                validate_parameters=False, verbosity=None)),
 ('LGBMClassifier', LGBMClassifier(random_state=42))]

#### 1.Voting - soft voting

In [25]:
# voting 파라미터로 hard/soft 선택
# hard -> 최빈값, soft -> 평균

voting_clf = VotingClassifier(estimators = estimators, voting='soft', n_jobs=-1)
voting_clf.fit(X_train, y_train)
pred = voting_clf.predict_proba(X_valid)
test_score = log_loss(y_valid, pred)
print(test_score)

0.4054019107242492


**성능을 내기 위해 위에 Voting에 추가적으로 해볼 수 있다고 생각되는 것들**
 - voting에 들어가는 모델들이 알고리즘적으로 다른 계열의 모델들을 사용함(logistic regression, knn, mlp, svc 등등)
 - 모델들을 더 많이 넣어 voting한다고 해서 성능이 개선되지는 않음, 성능이 좋으면서 서로 보완할 수 있을 것 같은 모델들을 앙상블 해야함
 - voting에 들어가는 모델들을 튜닝하여 좋은 성능이 나오는 모델을 voting에 사용함
 - voting_clf로 fit(학습)을 시킬 때 cross_validation방법을 사용함
 
 
모든 방법론에 있어서 이렇게 하는게 좋다 저렇게 하는게 좋다 같은 정답은 없음  
결국 다양한 시도를 통해 성능을 통해 감을 잡고 적당한 방법론을 택해야 함  
또한 꼭 앙상블한다고 해서 성능이 오르는 것은 아님, 오히려 싱글모델이 나을 때도 있음  

#### 2. Stacking

 - 스태킹은 sklearn, mlxtend, vecstack 등 다양한 패키지에서 제공함.  
다들 사용법이 비슷하며 해당 코드에서는 이해하기 쉬운 sklearn의 패키지를 사용하기로 함.  
참고로 조윤호 교수님은 vecstack을 이용함. 패키지마다의 속도 차이도 존재.  
 - voting보다 stacking이 더 오래 걸림(현 스크립트 기준)

In [7]:
# ?을 통해서 패키지 내부의 파라미터 인자 확인 가능
# ex) StackingClassifier?

In [26]:
estimators = [('rf', rf_clf), ('xgb', xgb_clf)]

In [27]:
stk_clf = StackingClassifier(estimators = estimators,
                            final_estimator = lgb_clf, cv=skf, n_jobs=-1)

stk_clf.fit(X_train, y_train)
pred = stk_clf.predict_proba(X_valid)
test_score = log_loss(y_valid, pred)
print(test_score)

0.3992179201696497


**성능을 내기 위해 위에 Stacking에 추가적으로 해볼 수 있다고 생각되는 것들**
 - 위에 voting 성능을 위해 적어 놓은 것들
 - meta 모델에는 사용했던 모델을 또 넣어도 되고 다른 모델을 넣어도 됨. 정답은 없음
 - vecstack를 이용하면 meta_clf를 stk_clf 내부에서 설정하지 않고 따로 외부에서 정의함. 이때 meta_clf를 튜닝하여 사용할 수 있겠음. (메타 모델 튜닝의 방법론)
 - meta 모델에 대한 cross_validation

#### 2 Layer Stacking
 - 이중 스태킹 : meta 모델에 단일 모델이 아닌 앙상블 모델을 넣어 두번 앙상블한다

     - 스태킹을 두번 하는 경우 예시

In [28]:
# 먼저 두번째에 들어갈 스태킹 층을 만들어줌, 최종적으로 메타모델이 될 모델은 gbm_clf
stk_layer2 = StackingClassifier(estimators = [('extra', extra_clf), ('lgb', lgb_clf)],
                            final_estimator = gbm_clf, cv=skf, n_jobs=-1)

# 메타 모델에 앞서 만든 두번 째 층 스태킹을 넣어줌
stk_clf = StackingClassifier(estimators = [('rf', rf_clf), ('xgb', xgb_clf)],
                            final_estimator = stk_layer2, cv=skf, n_jobs=-1)


stk_clf.fit(X_train, y_train)
pred = stk_clf.predict_proba(X_valid)

In [29]:
test_score = log_loss(y_valid, pred)
print(test_score)

0.39809573171689083


     - 스태킹 후 보팅을 하는 경우

In [30]:
voting_layer2 = VotingClassifier(estimators = [('extra', extra_clf), ('lgb', lgb_clf)], voting='soft', n_jobs=-1)
    # 앞에서 훈련한 보팅 모델을 가져다 쓰는 방법도 있음

stk_clf = StackingClassifier(estimators = [('rf', rf_clf), ('xgb', xgb_clf)],
                            final_estimator = voting_layer2, cv=skf, n_jobs=-1)

stk_clf.fit(X_train, y_train)
pred = stk_clf.predict_proba(X_valid)
test_score = log_loss(y_valid, pred)
print(test_score)

0.41912649143108893


## Submission Ensemble
 - 아래 코드는 예시임
 - 해당 서브미션 앙상블 또한 정답은 없으며 여러 시도를 거쳐서 어떤 방법이 성능이 잘 나오는지 테스크마다 확인해야함

In [31]:
sub_1 = pd.Series(np.random.ranf(5))
sub_2 = pd.Series(np.random.ranf(5))
sub_3 = pd.Series(np.random.ranf(5))

subs = [sub_1, sub_2, sub_3]

# 3개의 서브미션이 아래와 같이 찍혔다고 가정
# 전제 조건1 : 3개의 서브미션이 비슷한 방법이 아닌 각자 다른 방법으로 찍힘
# 전제 조건2 : 3개의 서브미션의 성능이 서로 비슷함
display(sub_1)
display(sub_2)
display(sub_3)

0    0.970217
1    0.709358
2    0.318606
3    0.543408
4    0.945127
dtype: float64

0    0.891055
1    0.245481
2    0.672198
3    0.807093
4    0.675388
dtype: float64

0    0.671197
1    0.489294
2    0.555514
3    0.708393
4    0.293071
dtype: float64

In [32]:
# 산술평균
final_sub = sum(subs) / len(subs)
final_sub

0    0.844157
1    0.481378
2    0.515439
3    0.686298
4    0.637862
dtype: float64

In [33]:
# 기하평균
from scipy.stats.mstats import gmean
final_sub = gmean(subs)
final_sub

array([0.83408046, 0.44003197, 0.4918306 , 0.67728997, 0.57192467])

In [34]:
# 가중평균
final_sub = (sub_1 * 0.5) + (sub_2 * 0.25) + (sub_3 * 0.25)
final_sub

# 그 외 조화평균, 멱평균 등의 방법들도 있다.

0    0.875672
1    0.538373
2    0.466231
3    0.650576
4    0.714678
dtype: float64

## **────────────────────────End of Pipeline──────────────────────**